라이브러리 버전<br>
판다스 버전 : 1.3.4

In [1]:
# 라이브러리 버전이 맞지 않을 경우 아래 명령어의 주석을 제거한 후 실행해주세요.
# 이전 예제들을 실행했다면 앞에서 이미 버전을 맞췄습니다.
# !pip install pandas==1.3.4

In [2]:
# 버전 확인용
import pandas as pd

print("판다스 버전 :", pd.__version__)

판다스 버전 : 1.3.4


필요한 라이브러리 불러오기

In [3]:
import pandas as pd
import os
import re
from ast import literal_eval

#시각화
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font',family='Malgun Gothic')

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

C:\Users\it\AppData\Local\Temp\ipykernel_16220\803475683.py:12: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')


In [4]:
basic_info_path = "./data/makeup_basic_info/"
df_bsc_rslt = pd.read_excel(basic_info_path + 'basic_info_result.xlsx').iloc[:,1:]

In [5]:
df_bsc_rslt.head(2)

,제품기호,dates,HashTag,likes,num_review,num_photo,year,month
0,0번_제품,2022.01.,#촉촉한수분감,4203,50,20,2022,1
1,0번_제품,2022.01.,#촉촉한사용감,4203,50,20,2022,1


In [6]:
df_bsc_rslt.tail(2)

,제품기호,dates,HashTag,likes,num_review,num_photo,year,month
8397,9번_제품,2020.10.,#글로시한느낌,13,1,1,2020,10
8398,9번_제품,2020.10.,#출근부터밤약속까지,13,1,1,2020,10


Rank() 함수로 해시태그에 가중치 더해주기

In [7]:
likes_rank = df_bsc_rslt['likes'].rank(method='dense')
review_rank = df_bsc_rslt['num_review'].rank(method='dense')
photo_rank = df_bsc_rslt['num_photo'].rank(method='dense')

rank = likes_rank + review_rank + photo_rank
df_bsc_rslt['rank'] = review_rank

df_bsc_rslt.sample(2)

,제품기호,dates,HashTag,likes,num_review,num_photo,year,month,rank
8,0번_제품,2022.01.,#20대여성,4203,50,20,2022,1,51.0
6339,76번_제품,2021.12.,#윤기있는,316,37,13,2021,12,38.0


In [8]:
import numpy as np

df_bsc_rslt['weight']=pd.cut(df_bsc_rslt['rank'], 10, labels=np.arange(1, 11))
df_bsc_rslt.sample(2)

,제품기호,dates,HashTag,likes,num_review,num_photo,year,month,rank,weight
3638,479번_제품,2020.11.,#보송보송한쿠션,164,2,2,2020,11,3.0,1
8070,939번_제품,2022.07.,#건강한피부,42,0,0,2022,7,1.0,1


In [9]:
df_bsc_rslt['weight'] = df_bsc_rslt['weight'].astype('int')

df_tag_2019  = df_bsc_rslt[df_bsc_rslt['year']==2019]
df_tag_2020  = df_bsc_rslt[df_bsc_rslt['year']==2020]
df_tag_2021  = df_bsc_rslt[df_bsc_rslt['year']==2021]

In [10]:
tag_2019 = list(df_tag_2019['HashTag']*df_tag_2019['weight'])
tag_2019 = "".join(tag_2019).split('#')

tag_2020 = list(df_tag_2020['HashTag']*df_tag_2020['weight'])
tag_2020 = "".join(tag_2020).split('#')

tag_2021 = list(df_tag_2021['HashTag']*df_tag_2021['weight'])
tag_2021 = "".join(tag_2021).split('#')

In [11]:
tag_2019[-3:]

['앰플커버', '앰플커버', '앰플커버']

해시태그 빈도 및 기울기 구하기

In [12]:
from collections import Counter

In [13]:
df_tag_2019 = pd.DataFrame(Counter(tag_2019).most_common(), columns=['해시태그','빈도_19년'])
df_tag_2020 = pd.DataFrame(Counter(tag_2020).most_common(), columns=['해시태그','빈도_20년'])
df_tag_2021 = pd.DataFrame(Counter(tag_2021).most_common(), columns=['해시태그','빈도_21년'])

In [14]:
prcnt_19y = (df_tag_2019['빈도_19년']/df_tag_2019['빈도_19년'].sum())*100
prcnt_20y = (df_tag_2020['빈도_20년']/df_tag_2020['빈도_20년'].sum())*100
prcnt_21y = (df_tag_2021['빈도_21년']/df_tag_2021['빈도_21년'].sum())*100
df_tag_2019['빈도(%)_19년']=np.round(prcnt_19y, 2)
df_tag_2020['빈도(%)_20년']=np.round(prcnt_20y, 2)
df_tag_2021['빈도(%)_21년']=np.round(prcnt_21y, 2)
df_tag_1920 = pd.merge(df_tag_2019, df_tag_2020, on='해시태그', how='outer')
df_tag_1921 = pd.merge(df_tag_1920, df_tag_2021, on='해시태그', how='outer')

df_tag_1921[['해시태그','빈도(%)_19년', '빈도(%)_20년', '빈도(%)_21년']][:20].sort_values(by='빈도(%)_21년', ascending=False)

,해시태그,빈도(%)_19년,빈도(%)_20년,빈도(%)_21년
6,지속력좋은,1.67,1.99,2.85
1,촉촉한쿠션,2.91,2.17,2.21
11,인생템,1.30,2.22,1.78
13,쿠션파운데이션,1.30,1.11,1.52
15,보송보송한메이크업,1.24,0.83,1.44
9,파운데이션인생템,1.36,1.67,1.40
0,커버력좋은쿠션,2.97,0.66,1.18
8,보송보송한쿠션,1.42,1.16,1.10
4,윤기커버,1.92,2.35,1.07
10,화사한파운데이션,1.30,0.33,1.00


In [15]:
slopes =[]
for i, j, k in zip(df_tag_1921['빈도(%)_19년'],df_tag_1921['빈도(%)_20년'],df_tag_1921['빈도(%)_21년']):
    try:
        slope = np.polyfit([i,j,k], [19,20,21], 1)
        slopes.append(slope[0])
    except:
        slopes.append(0)


df_tag_1921['기울기'] = slopes

In [16]:
df_tag_1921['빈도합계'] = df_tag_1921['빈도_19년'] + df_tag_1921['빈도_20년'] + df_tag_1921['빈도_21년']

In [17]:
df_tag_1921[(df_tag_1921['빈도합계']>=50) & (df_tag_1921['기울기']>0)].sort_values(by='기울기', ascending=False)

,해시태그,빈도_19년,빈도(%)_19년,빈도_20년,빈도(%)_20년,빈도_21년,빈도(%)_21년,기울기,빈도합계
114,광채피부,5.0,0.31,18.0,0.45,44.0,0.47,1.052632e+01,67.0
105,자연스러운,6.0,0.37,12.0,0.30,47.0,0.51,6.122449e+00,65.0
131,촉촉한선쿠션,3.0,0.19,13.0,0.33,48.0,0.52,6.014581e+00,64.0
55,보송보송한,9.0,0.56,27.0,0.68,87.0,0.94,5.035336e+00,123.0
51,파운데이션쿠션,10.0,0.62,20.0,0.51,71.0,0.76,4.458599e+00,101.0
118,드라마틱한,5.0,0.31,22.0,0.56,71.0,0.76,4.426230e+00,98.0
77,자연스런,8.0,0.49,33.0,0.83,71.0,0.76,4.188211e+00,112.0
113,화사한얼굴,5.0,0.31,31.0,0.78,80.0,0.86,3.114383e+00,116.0
124,생기있는,4.0,0.25,22.0,0.56,36.0,0.39,2.904564e+00,62.0
13,쿠션파운데이션,21.0,1.30,44.0,1.11,141.0,1.52,2.612827e+00,206.0


In [18]:
df_bsc_rslt.sort_values(by='rank', ascending=False).drop_duplicates(subset='제품기호')[:10]

,제품기호,dates,HashTag,likes,num_review,num_photo,year,month,rank,weight
7770,907번_제품,2021.03.,#커버쿠션,24698,50285,31668,2021,3,258.0,10
5069,63번_제품,2020.11.,#얼어죽어도매트,12792,18547,10981,2020,11,257.0,10
4395,564번_제품,2021.09.,#촉촉한팩트,86717,12804,6160,2021,9,256.0,10
5339,670번_제품,2017.05.,#수분팩트,11250,12239,1468,2017,5,255.0,10
5532,68번_제품,2021.03.,#쿠션추천,12792,6683,3687,2021,3,254.0,10
8175,94번_제품,2017.05.,#물광쿠션,10517,5822,281,2017,5,253.0,10
698,174번_제품,2017.12.,#건성쿠션,11250,5729,1131,2017,12,252.0,10
3475,460번_제품,2020.02.,#광채가빛나는,42085,5688,2967,2020,2,251.0,10
2306,337번_제품,2018.04.,#톤보정,26055,5595,516,2018,4,250.0,10
2073,312번_제품,2021.05.,#도자기피부,12722,5115,3012,2021,5,249.0,10


주요 제품 10개 찾아보기

In [19]:
df_top10 = df_bsc_rslt[df_bsc_rslt['제품기호'].str.contains('907번_제품') |\
                       df_bsc_rslt['제품기호'].str.contains('63번_제품')|\
                       df_bsc_rslt['제품기호'].str.contains('564번_제품') |\
                       df_bsc_rslt['제품기호'].str.contains('670번_제품')|\
                       df_bsc_rslt['제품기호'].str.contains('68번_제품') |\
                       df_bsc_rslt['제품기호'].str.contains('94번_제품')|\
                       df_bsc_rslt['제품기호'].str.contains('174번_제품') |\
                       df_bsc_rslt['제품기호'].str.contains('460번_제품')|\
                       df_bsc_rslt['제품기호'].str.contains('337번_제품') |\
                       df_bsc_rslt['제품기호'].str.contains('312번_제품')]

In [20]:
Counter(df_top10['HashTag'].tolist()).most_common(10)

[('#촉촉한쿠션', 10),
 ('#지속력좋은', 8),
 ('#생기넘치는피부', 7),
 ('#지속력좋은파운데이션', 7),
 ('#광채가빛나는', 6),
 ('#출근부터밤약속까지', 6),
 ('#화사한쿠션', 6),
 ('#투명하게빛나는피부', 5),
 ('#빛나는피부', 5),
 ('#윤기나는얼굴', 5)]